In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm

In [ ]:
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
dataset_root = 'cifar100_data'
trainset = datasets.CIFAR100(
    dataset_root,
    train=True,
    transform=data_transforms,
    download=True
)

testset = datasets.CIFAR100(
    dataset_root,
    train=False,
    transform=data_transforms,
    download=True
)

100%|██████████| 169001437/169001437 [00:02<00:00, 67727409.97it/s]


Extracting cifar100_data/cifar-100-python.tar.gz to cifar100_data
Files already downloaded and verified


In [ ]:
training_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testing_loader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

In [ ]:
class MyCustomCNN(nn.Module):
    def __init__(self):
        super(MyCustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 2 * 2, 512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(512, 100)  # 100 classes for CIFAR-100

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)

        out = self.conv3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)


        out = torch.flatten(out, 1)

        out = self.fc1(out)
        out = self.relu4(out)
        out = self.fc2(out)
        return out

In [ ]:
num_epochs = 50 #should be at least 30
model = MyCustomCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for step, (images, labels) in enumerate(tqdm(training_loader)):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    scheduler.step()

torch.save(model.state_dict(), 'best_model.pth')


100%|██████████| 1563/1563 [00:20<00:00, 77.71it/s]


In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [ ]:
model.eval()
total = 0
correct = 0
with torch.no_grad():
    for step, (images, labels) in enumerate(tqdm(testing_loader)):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total

print(f'Epoch [{epoch+1}/{num_epochs}]: Accuracy = {accuracy * 100:.2f}%')

100%|██████████| 313/313 [00:03<00:00, 83.80it/s]

Epoch [50/50]: Accuracy = 45.92%
